## PDF Dateien

- [PDF](https://de.wikipedia.org/wiki/Portable_Document_Format) Dateien beinhalten z.B. Bilder oder Seitenumbrüche  und sollten nur dann verarbeitet werden, wenn keine andere Möglichkeit besteht.
- Wenn es um das scrapen von Tabellen geht, die z.B: in mehreren PDFs immer an der gleichen Stelle stehen, bietet sich das Tool [tabula](http://tabula.technology/) an.
- Für andere Anwendungsfälle muss die PDF Datei meistens in Rohtext konvertiert und anschließend über reguläre Ausdrücke verarbeitet werden.

### Beispiel: Reviewer eines Journals

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import IFrame

In [ ]:
IFrame('http://ajps.org/list-of-reviewers/', width = 800, height = 400)

Zunächst lesen wir die Website ein und parsen mit BeautifulSoup:

In [ ]:
base = "http://ajps.org/list-of-reviewers/"
r = requests.get(base)
soup = BeautifulSoup(r.text,  'html5lib')

In [ ]:
links = [link.get('href') for link in soup.find_all('a')]
links[:10]

In [ ]:
reviews = [ i  for i in links if i != None and re.search('reviewer.*?\.pdf', i)]
reviews

Im Gegensatz zu beispielsweise `JSON` Daten sollten PDFs (ebenso wie Bilder und Video)  nicht in Textform, sondern in Binärform abgespeichert werden:

In [ ]:
%cd "D:\datascraping\data"

In [ ]:
p = requests.get('https://ajpsblogging.files.wordpress.com/2014/01/ajps_reviewers_2013.pdf')

Ausgabe des Textes einer Binärdatei (nicht besonders nützlich):

In [ ]:
p.text[:200]

File abspeichern:

In [ ]:
with open('example.pdf', 'wb') as f: # wb für write binary
    f.write(p.content) # content anstelle von text

### Übungsaufgabe 1

Speichert alle Reviewer PDFs lokal auf euren Rechnern. Die entsprechenden Files sollen dabei alle nach dem Muster `ajps_reviewer_jahr.pdf` benannt werden.    

In [ ]:
# Code für Übungsaufgabe 1


In [ ]:
%cd "D:\Dropbox\lehre\Data Scraping\data"

Nach dem herunterladen PDF der Dateien kann das Paket [PDFMiner](https://github.com/pdfminer/pdfminer.six) verwendet werden, um  PDFs in Rohtext zu konvertieren.

Installation falls notwendig:

In [ ]:
!pip install pdfminer.six

Importe und Funktionsdefinition:

In [ ]:
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [ ]:
# https://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library
def convert_pdf_to_txt(path):
    '''function for converting a .pdf file to text'''
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages,
                                  password = password,
                                  caching = caching,
                                  check_extractable = True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

 Wir arbeiten exemplarisch mit der PDF-Version für das Jahr 2013:

In [ ]:
text = convert_pdf_to_txt("ajps_reviewer_2013.pdf")

In [ ]:
text[:500]

In [ ]:
print(text[:500])

In [ ]:
text[-500:]

In [ ]:
print(text[-500:])

Wir entfernen zunächst die unnötigen `!` sowie Zeilenumbrüche und zerlegen den Text anschließend zeilenweise.

In [ ]:
to_replace = ['!', '\x0c', '  ']
for string in to_replace:
    text = text.replace(string, ' ')
print(text[-700:])

In [ ]:
text.split('\n')[:20]

In [ ]:
lines = [line for line in text.split('\n')[13:] # Preambel entfernen
         if len(line) > 2]
lines[:5]

### Übungsaufgabe 2

Versucht die Review Daten weiter aufzubereiten und anschließend in einem Datensatz mit den Variablen `name`, `institution` und `review_count`zu speichern.

Beispiel - Inputzeile: 

` 'Deniz Aksoy , Princeton University ( 1 ) ' `

Beispiel - Ouput im Datensatz:    
`` name: "Deniz Aksoy", institution: "Princeton University", review_count: 1 ``


In [ ]:
# Code Übungsaufgabe 2

[Exkurs] Die Reviewer Informationen könnten beispielsweise zur Visualisierung der Institutionsstandorte verwendet werden. Nützliche Pakete für Geocoding:
-  [geopy](https://geopy.readthedocs.io/en/1.10.0/): Zugriff auf Geocoding durch externe APIs
-  [Folium](https://folium.readthedocs.io/en/latest/): Darstellung von Geolocations in interaktiven Karten 

## Browserautomatisierung mit Selenium

- Es gibt viele Webseiten die sich nicht vollständig über einfaches HTML Scraping verarbeiten lassen:
    - Seiten die über Javascript die HTML Struktur verändern
    - Seiten die einen User Input, also z.B. für eine Suchmaske, benötigen
- Für solche Fälle kann [Selenium](http://selenium-python.readthedocs.io/) verwendet werden, um einen Browser zu automatisieren

In [ ]:
from selenium import webdriver # ggf selenium ueber pip oder conda installieren
from bs4 import BeautifulSoup
import os
import re

Um mit Selenium zu arbeiten muss sich eine mit eurem Chrome Browser kompatible Version des [chromedrivers](http://chromedriver.chromium.org/downloads)  im Arbeitsverzeichnis  befinden.

In [ ]:
%cd "D:\Selenium"

Ordner über das `os` Paket erstellen falls nicht vorhanden:

In [ ]:
cwd =  os.getcwd()
if not os.path.exists(str(cwd) + "/data"):
    os.mkdir(str(cwd) + "/data")

Download Ordner des Chrome Browsers anpassen:

In [ ]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : os.path.join(str(cwd),"/data")}
chromeOptions.add_experimental_option("prefs",prefs)

Browser Verbindung herstellen:

In [ ]:
browser = webdriver.Chrome(executable_path = os.path.join(os.getcwd(), "chromedriver"), 
                           options=chromeOptions) 

Webseite ansteuern:

In [ ]:
browser.get("https://duckduckgo.com/")

Elemente im Browser können über verschiedene Methoden (name, klasse, css selektor, id, ..) ausfindig gemacht werden:

In [ ]:
search = browser.find_element_by_name('q')
search = browser.find_element_by_css_selector('#search_form_input_homepage')

Je nach Typ des Elements können anschließend Clicks, Tastatureingaben, oder z.B. Downloads gestartet werden:

In [ ]:
search.send_keys('The night is dark and full of terrors')

In [ ]:
search.clear()

Strings können dabei wie üblich in Python dynamisch generiert werden:

In [ ]:
year = 2020
search.send_keys('BlackLivesMatter {}'.format(year))

Die Suche über einen simulierten Mausklick abschicken:

In [ ]:
search_button = browser.find_element_by_id('search_button_homepage')
search_button.click()

Es können auch Screenshots vom aktuellen Stand des Browserinhalts gemacht werden:

In [ ]:
browser.get_screenshot_as_file('data/screenshot_searchengine.png')

Ebenso kann der aktuelle HTMl Code aus dem Browser ausgelesen werden (z.B: nachdem JavaScript Code die Inhalte verändert hat):

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html, 'html5lib')
soup.find_all('a')[:5]

Weitere Informationen zur Verwendung von Selenium finden sich auf der entsprechenden [Webseite](https://www.seleniumhq.org/). Für umfangreiche Browserautomatisierungen bietet sich zudem das auf Selenium aufbauende Tool [OpenWPM](https://github.com/mozilla/OpenWPM) an.

<br>
<br>


___

                
**Kontakt: Carsten Schwemmer** (Webseite: www.carstenschwemmer.com,  Email: c.schwem2er@gmail.com)